<a href="https://colab.research.google.com/github/napstruck/DetectTheAudio/blob/branch-1/Major_Project_FL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation and Data set extraction

In [1]:
# prompt: Install all tensorflow-federated and librosa that is compatible with now version of google colab

!pip install tensorflow-federated
!pip install librosa


In [2]:
!pip install --upgrade tensorflow-federated
import tensorflow as tf
import tensorflow_federated as tff

ERROR:jax._src.xla_bridge:Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/xla_bridge.py", line 438, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/usr/local/lib/python3.11/dist-packages/jax_plugins/xla_cuda12/__init__.py", line 85, in initialize
    options = xla_client.generate_pjrt_gpu_plugin_options()
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: module 'jaxlib.xla_client' has no attribute 'generate_pjrt_gpu_plugin_options'


In [3]:
!pip install --upgrade pip setuptools wheel

In [4]:
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np
import librosa
import os
from tensorflow.keras.layers import Input, Concatenate, Dense, Flatten


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import zipfile
import os

# Path to the zip file
zip_path = "/content/drive/MyDrive/audio_dataset.zip"

# Directory to extract the dataset
extract_path = "/content/dataset"

# Create the extraction directory
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")

Dataset extracted successfully!


In [7]:
dataset_path = os.path.join(extract_path, "audio_dataset")

In [8]:
import tensorflow_federated as tff
print(tff.__version__)


0.87.0


In [9]:
pip install tensorflow-federated==0.18.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 67.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.3/887.3 kB 32.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tensorflow-federated to determine which version is compatible with other requirements. This could take a while.
ERROR: Ignored the following yanked versions: 0.4.32
ERROR: Ignored the following versions that require a different python version: 0.34.0 Requires-Python ~=3.9.0; 0.36.0 Requires-Python ~=3.9.0; 0.37.0 Requires-Python >=3.9.0,<3.11; 0.38.0 Requires-Python >=3.9.0,<3.11; 0.39.0 Requires-Python >=3.9.0,<3.11; 0.40.0 Requires-Python >=3.9.0,<3.11; 0.41.0 Requires-Python >=3.9.0,<3.11; 0.42.0 Requires-Python >=3.9.0,<3.11; 0.43.0 Requires-Python >=3.9.0,<3.11; 0.44.0 Requires-Python >=3.9.0,<3.11; 0.45.0 Requires-Python >=3.9.0,<3.11; 0.46.0 Requires-Py

In [10]:
print(dir(tff.learning.optimizers))


['LEARNING_RATE_KEY', 'Optimizer', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'adafactor', 'adagrad', 'adam', 'adamw', 'build_adafactor', 'build_adagrad', 'build_adam', 'build_adamw', 'build_rmsprop', 'build_sgdm', 'build_yogi', 'check_weights_gradients_match', 'handle_indexed_slices_gradients', 'keras_optimizer', 'nest_utils', 'optimizer', 'rmsprop', 'schedule_learning_rate', 'scheduling', 'sgdm', 'yogi']


**Final trial of the FL code using 1st method!!**

In [14]:
# import tensorflow as tf
# import tensorflow_federated as tff
# import librosa
# import os

# # Feature extraction function
# def extract_features(audio, sample_rate=16000):
#     audio = audio.numpy()
#     features = {
#         'mfccs': librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13).T,
#         'chroma': librosa.feature.chroma_stft(y=audio, sr=sample_rate).T,
#         'spectral_contrast': librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T,
#         'zero_crossing_rate': librosa.feature.zero_crossing_rate(audio).T,
#         'rms_energy': librosa.feature.rms(y=audio).T,
#         'mel_spectrogram': librosa.feature.melspectrogram(y=audio, sr=sample_rate).T
#     }
#     return features

# # Preprocessing function
# def preprocess_audio(file_path, label):
#     audio = tf.io.read_file(file_path)
#     audio, _ = tf.audio.decode_wav(audio, desired_channels=1)
#     audio = tf.squeeze(audio, axis=-1)
#     features = tf.py_function(
#         extract_features, [audio],
#         Tout=[tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32]
#     )
#     return {
#         'mfccs': features[0],
#         'chroma': features[1],
#         'spectral_contrast': features[2],
#         'zero_crossing_rate': features[3],
#         'rms_energy': features[4],
#         'mel_spectrogram': features[5]
#     }, label

# # Dataset preparation
# dataset_path = "/content/dataset/audio_dataset"  # Replace with your dataset path
# fake_files = tf.data.Dataset.list_files(os.path.join(dataset_path, "fake/*.wav"), shuffle=False)
# real_files = tf.data.Dataset.list_files(os.path.join(dataset_path, "real/*.wav"), shuffle=False)

# fake_dataset = fake_files.map(lambda x: (x, 0))  # 0 for fake
# real_dataset = real_files.map(lambda x: (x, 1))  # 1 for real

# full_dataset = fake_dataset.concatenate(real_dataset).map(preprocess_audio)
# full_dataset = full_dataset.shuffle(buffer_size=88284, seed=42)

# dataset_size = 88284
# chunk_size = dataset_size // 6
# chunks = [full_dataset.skip(i * chunk_size).take(chunk_size) for i in range(6)]
# train_clients_data = chunks[:5]
# test_data = chunks[5]

# # Federated learning setup
# def create_federated_model():
#     keras_model = tf.keras.Sequential([
#         tf.keras.layers.InputLayer(input_shape=(13,)),  # Adjust input shape based on your features
#         tf.keras.layers.Dense(64, activation='relu'),
#         tf.keras.layers.Dense(64, activation='relu'),
#         tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification (fake/real)
#     ])
#     return tff.learning.from_keras_model(
#         keras_model,
#         input_spec=(
#             tf.TensorSpec(shape=[None, 13], dtype=tf.float32),  # Adjust input spec based on your features
#             tf.TensorSpec(shape=[None], dtype=tf.int32)
#         ),
#         loss=tf.keras.losses.BinaryCrossentropy(),
#         metrics=[tf.keras.metrics.BinaryAccuracy()]
#     )

# # Define client and server optimizers as instances (not functions)
# client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.02)
# server_optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# # Build the iterative process
# iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
#     model_fn=create_federated_model,
#     client_optimizer_fn=lambda: client_optimizer,
#     server_optimizer_fn=lambda: server_optimizer
# )

# print("Federated learning process created successfully! 🚀")

AttributeError: module 'tensorflow_federated.python.learning' has no attribute 'from_keras_model'

In [16]:
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np

# Load dataset (MNIST)
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
x_train, y_train = x_train / 255.0, y_train  # Normalize

# Split data into 6 parts for federated clients
NUM_CLIENTS = 6
client_datasets = []
split_size = len(x_train) // NUM_CLIENTS

for i in range(NUM_CLIENTS):
    start, end = i * split_size, (i + 1) * split_size
    client_datasets.append(
        tf.data.Dataset.from_tensor_slices((x_train[start:end], y_train[start:end]))
        .shuffle(100)
        .batch(20)
    )

# Create a Keras model
def create_keras_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

# Convert Keras model to a TFF model
class TFFModel(tff.learning.models.Model):
    def __init__(self):
        self._keras_model = create_keras_model()
        self._loss = tf.keras.losses.SparseCategoricalCrossentropy()
        self._metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

    @property
    def trainable_variables(self):
        return self._keras_model.trainable_variables

    @property
    def non_trainable_variables(self):
        return self._keras_model.non_trainable_variables

    @property
    def local_variables(self):
        return [tf.Variable(0.0, name='loss')]

    @property
    def input_spec(self):
        return client_datasets[0].element_spec  # Define input format

    @tf.function
    def forward_pass(self, batch, training=True):
        predictions = self._keras_model(batch[0], training=training)
        loss = self._loss(batch[1], predictions)
        self.local_variables[0].assign(loss)
        return tff.learning.models.BatchOutput(
            loss=loss, predictions=predictions, num_examples=tf.shape(batch[1])[0]
        )

    @tf.function
    def report_local_unfinalized_metrics(self):
        return [self.local_variables[0]]

    def metric_finalizers(self):
        return [tf.function(lambda x: x)]

    def federated_output_computation(self):
        return tff.learning.metrics.sum_then_finalize

# Function to create the TFF model
def model_fn():
    return TFFModel()

# Optimizers
def client_optimizer_fn():
    return tf.keras.optimizers.SGD(learning_rate=0.02)

def server_optimizer_fn():
    return tf.keras.optimizers.SGD(learning_rate=1.0)

# Build Federated Averaging Process
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=model_fn,
    client_optimizer_fn=client_optimizer_fn,
    server_optimizer_fn=server_optimizer_fn
)

# Initialize model
state = iterative_process.initialize()

# Run Federated Training for 10 rounds
for round_num in range(10):
    state, metrics = iterative_process.next(state, client_datasets)
    print(f'Round {round_num + 1}, Metrics: {metrics}')


AttributeError: module 'tensorflow_federated.python.learning.models' has no attribute 'Model'

In [8]:
# def extract_features(audio, sample_rate=16000):
#     audio = audio.numpy()
#     features = {
#         'mfccs': librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13).T,
#         'chroma': librosa.feature.chroma_stft(y=audio, sr=sample_rate).T,
#         'spectral_contrast': librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T,
#         'zero_crossing_rate': librosa.feature.zero_crossing_rate(audio).T,
#         'rms_energy': librosa.feature.rms(y=audio).T,
#         'mel_spectrogram': librosa.feature.melspectrogram(y=audio, sr=sample_rate).T
#     }
#     return features

In [9]:
# def preprocess_audio(file_path, label):
#     audio = tf.io.read_file(file_path)
#     audio, _ = tf.audio.decode_wav(audio, desired_channels=1)
#     audio = tf.squeeze(audio, axis=-1)
#     features = tf.py_function(extract_features, [audio],
#                               Tout=[tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32])
#     return { 'mfccs': features[0], 'chroma': features[1], 'spectral_contrast': features[2],
#              'zero_crossing_rate': features[3], 'rms_energy': features[4], 'mel_spectrogram': features[5] }, label


In [24]:
# # Your code to prepare the dataset
# fake_files = tf.data.Dataset.list_files(os.path.join(dataset_path, "fake/*.wav"), shuffle=False)
# real_files = tf.data.Dataset.list_files(os.path.join(dataset_path, "real/*.wav"), shuffle=False)

# fake_dataset = fake_files.map(lambda x: (x, 0))  # 0 for fake
# real_dataset = real_files.map(lambda x: (x, 1))  # 1 for real

# full_dataset = fake_dataset.concatenate(real_dataset).map(preprocess_audio)
# full_dataset = full_dataset.shuffle(buffer_size=88284, seed=42)

# dataset_size = 88284
# chunk_size = dataset_size // 6
# chunks = [full_dataset.skip(i * chunk_size).take(chunk_size) for i in range(6)]
# train_clients_data = chunks[:5]
# test_data = chunks[5]

# # My code to set up federated learning
# def create_federated_model():
#     keras_model = tf.keras.Sequential([
#         tf.keras.layers.Dense(10, input_shape=(784,), activation='relu'),
#         tf.keras.layers.Dense(10, activation='softmax')
#     ])
#     return tff.learning.from_keras_model(
#         keras_model,
#         input_spec=(tf.TensorSpec(shape=[None, 784], dtype=tf.float32),
#                     tf.TensorSpec(shape=[None, 10], dtype=tf.int32)),
#         loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#         metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
#     )

# def client_optimizer_fn():
#     return tf.keras.optimizers.SGD(learning_rate=0.02)

# def server_optimizer_fn():
#     return tf.keras.optimizers.Adam(learning_rate=0.01)

# iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
#     model_fn=create_federated_model,
#     client_optimizer_fn=client_optimizer_fn,
#     server_optimizer_fn=server_optimizer_fn
# )

# # Now you can use `train_clients_data` as client datasets in federated learning

AttributeError: module 'tensorflow_federated.python.learning' has no attribute 'from_keras_model'

In [10]:
# fake_files = tf.data.Dataset.list_files(os.path.join(dataset_path, "fake/*.wav"), shuffle=False)
# real_files = tf.data.Dataset.list_files(os.path.join(dataset_path, "real/*.wav"), shuffle=False)


In [11]:
# fake_dataset = fake_files.map(lambda x: (x, 0))  # 0 for fake
# real_dataset = real_files.map(lambda x: (x, 1))  # 1 for real


In [12]:
# full_dataset = fake_dataset.concatenate(real_dataset).map(preprocess_audio)
# full_dataset = full_dataset.shuffle(buffer_size=88284, seed=42)


In [13]:
# dataset_size = 88284
# chunk_size = dataset_size // 6
# chunks = [full_dataset.skip(i * chunk_size).take(chunk_size) for i in range(6)]
# train_clients_data = chunks[:5]
# test_data = chunks[5]

In [14]:
# train_clients_data = chunks[:5]
# test_data = chunks[5]



In [23]:
# import tensorflow as tf
# import tensorflow_federated as tff

# # Example function to create a federated model
# def create_federated_model():
#     # Define a simple Keras model
#     keras_model = tf.keras.Sequential([
#         tf.keras.layers.Dense(10, input_shape=(784,), activation='relu'),
#         tf.keras.layers.Dense(10, activation='softmax')
#     ])

#     # Convert the Keras model to a TFF model
#     return tff.learning.from_keras_model(
#         keras_model,
#         input_spec=(tf.TensorSpec(shape=[None, 784], dtype=tf.float32),
#                     tf.TensorSpec(shape=[None, 10], dtype=tf.int32)),
#         loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#         metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
#     )

# # Define the client optimizer function
# def client_optimizer_fn():
#     return tf.keras.optimizers.SGD(learning_rate=0.02)

# # Define the server optimizer function
# def server_optimizer_fn():
#     return tf.keras.optimizers.Adam(learning_rate=0.01)

# # Build the iterative process for Federated Averaging
# iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
#     model_fn=create_federated_model,
#     client_optimizer_fn=client_optimizer_fn,
#     server_optimizer_fn=server_optimizer_fn
# )

# # Now you can use `iterative_process` to run federated learning

AttributeError: module 'tensorflow_federated.python.learning' has no attribute 'from_keras_model'

In [19]:
# def create_federated_model():
#     mfcc_input = Input(shape=(100, 13), name='mfccs')
#     chroma_input = Input(shape=(100, 12), name='chroma')
#     spectral_contrast_input = Input(shape=(100, 7), name='spectral_contrast')
#     zero_crossing_input = Input(shape=(100, 1), name='zero_crossing_rate')
#     rms_input = Input(shape=(100, 1), name='rms_energy')
#     mel_input = Input(shape=(100, 128), name='mel_spectrogram')

#     flattened_features = [Flatten()(mfcc_input), Flatten()(chroma_input), Flatten()(spectral_contrast_input),
#                           Flatten()(zero_crossing_input), Flatten()(rms_input), Flatten()(mel_input)]
#     concatenated = Concatenate()(flattened_features)
#     x = Dense(128, activation='relu')(concatenated)
#     x = Dense(64, activation='relu')(x)
#     output = Dense(1, activation='sigmoid')(x)

#     from tensorflow.keras.models import Model

#     keras_model = Model(inputs=[mfcc_input, chroma_input, spectral_contrast_input,
#                                 zero_crossing_input, rms_input, mel_input], outputs=output)

#     return tff.learning.models.from_keras_model(
#         keras_model,
#         input_spec=train_clients_data[0].element_spec,
#         loss=tf.keras.losses.BinaryCrossentropy(),
#         metrics=[tf.keras.metrics.BinaryAccuracy()]
#     )

In [20]:
# def client_optimizer_fn():
#     return lambda: tff.learning.optimizers.build_sgdm(learning_rate=0.001)
# def server_optimizer_fn():
#     return lambda: tff.learning.optimizers.build_sgdm(learning_rate=0.001)

<!--  -->

In [21]:
# iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
#     model_fn=create_federated_model,
#     client_optimizer_fn=client_optimizer_fn,
#     server_optimizer_fn=server_optimizer_fn
# )


AttributeError: 'function' object has no attribute 'initialize'

In [ ]:
# state = iterative_process.initialize()
# for round_num in range(10):
#     state, metrics = iterative_process.next(state, train_clients_data)
#     print(f"Round {round_num + 1}, Metrics: {metrics}")

In [ ]:
# def extract_features(audio, sample_rate=16000):
#     """
#     Extract multiple audio features from a waveform.
#     Args:
#         audio (tf.Tensor): Audio waveform (1D tensor).
#         sample_rate (int): Sample rate of the audio.
#     Returns:
#         features (dict): Dictionary of extracted features.
#     """
#     audio = audio.numpy()  # Convert TensorFlow tensor to NumPy array
#     features = {}

#     # 1. MFCCs
#     mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
#     features['mfccs'] = mfccs.T  # Transpose to match time steps

#     # 2. Chroma Features
#     chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
#     features['chroma'] = chroma.T

#     # 3. Spectral Contrast
#     spectral_contrast = librosa.feature.spectral_contrast(y=audio, sr=sample_rate)
#     features['spectral_contrast'] = spectral_contrast.T

#     # 4. Zero-Crossing Rate
#     zero_crossing_rate = librosa.feature.zero_crossing_rate(audio)
#     features['zero_crossing_rate'] = zero_crossing_rate.T

#     # 5. RMS Energy
#     rms_energy = librosa.feature.rms(y=audio)
#     features['rms_energy'] = rms_energy.T

#     # 6. Mel Spectrogram
#     mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
#     features['mel_spectrogram'] = mel_spectrogram.T

#     return features
